In [1]:
import numpy as np 
import pandas as pd 
import string 
import torch 
import torch.nn as nn 
from transformers import BertPreTrainedModel, BertModel, BertTokenizerFast, AdamW
from colbert.parameters import DEVICE 
import os 
import pickle

In [2]:
# check saved checkpoints 
os.listdir('experiments/dirty/train.py/2021-12-06_08.01.48/checkpoints/')

['colbert-30000.dnn',
 'colbert-32000.dnn',
 'colbert.dnn',
 'colbert-10000.dnn',
 'colbert-20000.dnn',
 'colbert-25000.dnn']

# Create Indexes For Corpus

In [22]:
!python -m colbert.index --amp --doc_maxlen 512 --query_maxlen 512 \
--mask-punctuation --bsize 256 --checkpoint ./experiments/dirty/train.py/2021-12-06_08.01.48/checkpoints/colbert-32000.dnn \
--collection test_collections.tsv --index_root ./experiments/indexes --index_name large_train_index

1 False 64


[Dec 07, 02:43:29] #> Creating directory /workspace/patent_similarity/ColBERT/experiments/dirty/index.py/2021-12-07_02.43.21 




[Dec 07, 02:43:29] #> Creating directory /workspace/patent_similarity/ColBERT/experiments/dirty/index.py/2021-12-07_02.43.21/logs/ 


[Dec 07, 02:43:29] {'root': 'experiments', 'experiment': 'dirty', 'run': '2021-12-07_02.43.21', 'rank': -1, 'similarity': 'cosine', 'dim': 128, 'query_maxlen': 512, 'doc_maxlen': 512, 'mask_punctuation': True, 'checkpoint': './experiments/dirty/train.py/2021-12-06_08.01.48/checkpoints/colbert-32000.dnn', 'bsize': 256, 'amp': True, 'collection': 'test_collections.tsv', 'index_root': './experiments/indexes', 'index_name': 'large_train_index', 'chunksize': 6.0} 



[Dec 07, 02:43:29] #> Note: Output directory ./experiments/indexes already exists




[Dec 07, 02:43:29] #> Creating directory ./experiments/indexes/large_train_index 


[Dec 07, 02:43:29] [0] 		 #> Local args.bsize = 256
[Dec 07, 02:43:29] [0] 		 #> args.

# FAISS Indexing for End-To-End Retrieval

In [23]:
!python -m colbert.index_faiss --index_root ./experiments/indexes --index_name large_train_index 

1 False 64


[Dec 07, 02:54:47] #> Creating directory /workspace/patent_similarity/ColBERT/experiments/dirty/index_faiss.py/2021-12-07_02.54.47 




[Dec 07, 02:54:47] #> Creating directory /workspace/patent_similarity/ColBERT/experiments/dirty/index_faiss.py/2021-12-07_02.54.47/logs/ 


[Dec 07, 02:54:47] {'root': 'experiments', 'experiment': 'dirty', 'run': '2021-12-07_02.54.47', 'rank': -1, 'index_root': './experiments/indexes', 'index_name': 'large_train_index', 'partitions': None, 'sample': None, 'slices': 1} 

#> num_embeddings = 38915192



[Dec 07, 02:54:47] [WARNING] 	 You did not specify --partitions!
[Dec 07, 02:54:47] [WARNING] 	 Default computation chooses 65536 partitions (for 38915192 embeddings)



[Dec 07, 02:54:47] #> Starting..
[Dec 07, 02:54:47] #> Processing slice #1 of 1 (range 0..2).
[Dec 07, 02:54:47] #> Will write to ./experiments/indexes/large_train_index/ivfpq.65536.faiss.
[Dec 07, 02:54:47] #> Loading ./experiments/indexes/large_train_index/0.sample ...
[Dec

# Retrieve and ReRank

There are 77886 documents in the corpus. 

There are 38943 queries. We test with a smaller query size (first 100 and first 500 respectively) due to time and memory constraints. 



In [3]:
test_queries = pd.read_csv("test_queries.tsv",sep="\t") 

test_queries.head(3)

,id,queries
0,0,A system for charging or maintaining a batter...
1,1,A method for calculating a path delay in stat...
2,2,A preventive or therapeutic agent for diabete...


In [9]:
small_test_queries = test_queries.iloc[:100,:] 

small_test_queries.to_csv("small_test_queries.tsv", index=False, sep="\t") 

In [10]:
test_collections = pd.read_csv("test_collections.tsv", sep="\t")  

test_collections.head(3)

,id,passages
0,0,A method for battery charger and diagnosis wi...
1,1,A non transitory computer readable medium car...
2,2,A method for measuring glutamyl transpeptidas...


In [11]:
small_test_queries.shape, test_collections.shape

((100, 2), (77886, 2))

In [12]:
!python -m colbert.retrieve --amp --doc_maxlen 512 --query_maxlen 512 --bsize 256 \
--queries small_test_queries.tsv --partitions 65536\
--index_root ./experiments/indexes --index_name large_train_index \
--collection test_collections.tsv \
--checkpoint ./experiments/dirty/train.py/2021-12-06_08.01.48/checkpoints/colbert-32000.dnn --batch --retrieve_only 

1 False 64


[Dec 08, 08:51:14] #> Creating directory /workspace/patent_similarity/ColBERT/experiments/dirty/retrieve.py/2021-12-08_08.51.06 




[Dec 08, 08:51:14] #> Creating directory /workspace/patent_similarity/ColBERT/experiments/dirty/retrieve.py/2021-12-08_08.51.06/logs/ 


[Dec 08, 08:51:14] {'root': 'experiments', 'experiment': 'dirty', 'run': '2021-12-08_08.51.06', 'rank': -1, 'similarity': 'cosine', 'dim': 128, 'query_maxlen': 512, 'doc_maxlen': 512, 'mask_punctuation': False, 'checkpoint': './experiments/dirty/train.py/2021-12-06_08.01.48/checkpoints/colbert-32000.dnn', 'bsize': 256, 'amp': True, 'queries': 'small_test_queries.tsv', 'collection': 'test_collections.tsv', 'qrels': None, 'index_root': './experiments/indexes', 'index_name': 'large_train_index', 'partitions': 65536, 'nprobe': 10, 'retrieve_only': True, 'faiss_name': None, 'faiss_depth': 1024, 'part_range': None, 'batch': True, 'depth': 1000} 

Some weights of the model checkpoint at bert-base-uncased were not u

In [14]:
!python -m colbert.rerank --batch --log-scores --topk ./experiments/dirty/retrieve.py/2021-12-08_08.51.06/unordered.tsv \
--query_maxlen 512 --doc_maxlen 512 --mask-punctuation \
--checkpoint ./experiments/dirty/train.py/2021-12-06_08.01.48/checkpoints/colbert-32000.dnn \
--amp --queries small_test_queries.tsv \
--index_root ./experiments/indexes --index_name large_train_index --bsize 8

1 False 64


[Dec 08, 09:22:33] #> Creating directory /workspace/patent_similarity/ColBERT/experiments/dirty/rerank.py/2021-12-08_09.22.24 




[Dec 08, 09:22:33] #> Creating directory /workspace/patent_similarity/ColBERT/experiments/dirty/rerank.py/2021-12-08_09.22.24/logs/ 


[Dec 08, 09:22:33] {'root': 'experiments', 'experiment': 'dirty', 'run': '2021-12-08_09.22.24', 'rank': -1, 'similarity': 'cosine', 'dim': 128, 'query_maxlen': 512, 'doc_maxlen': 512, 'mask_punctuation': True, 'checkpoint': './experiments/dirty/train.py/2021-12-06_08.01.48/checkpoints/colbert-32000.dnn', 'bsize': 8, 'amp': True, 'queries': 'small_test_queries.tsv', 'collection': None, 'qrels': None, 'topK': './experiments/dirty/retrieve.py/2021-12-08_08.51.06/unordered.tsv', 'shortcircuit': False, 'index_root': './experiments/indexes', 'index_name': 'large_train_index', 'partitions': None, 'step': 1, 'part_range': None, 'log_scores': True, 'batch': True, 'depth': 1000} 

Some weights of the model checkpoint at b

#> ( QID 0 ) 1)  71944 : 484.0531311035156      None
#> ( QID 0 ) 2)  48715 : 484.0531311035156      None



/workspace/patent_similarity/ColBERT/experiments/dirty/rerank.py/2021-12-08_09.22.24/ranking.tsv
[Dec 08, 09:29:12] #> Done.



# Evaluation: MRR (Mean Reciprocal Rank) for 100 queries

In [15]:
rankings = pd.read_csv("experiments/dirty/rerank.py/2021-12-08_09.22.24/ranking.tsv", sep="\t") 

rankings.head()

,0,71944,1,484.0531311035156
0,0,48715,2,484.053131
1,0,69480,3,484.053131
2,0,10795,4,483.817047
3,0,2853,5,483.521179
4,0,53253,6,483.255737


In [16]:
rankings = rankings.columns.to_frame().T.append(rankings, ignore_index=True) 

rankings.columns = range(len(rankings.columns)) 

rankings.head()

,0,1,2,3
0,0,71944,1,484.0531311035156
1,0,48715,2,484.053
2,0,69480,3,484.053
3,0,10795,4,483.817
4,0,2853,5,483.521


In [17]:
### load dictionary that contains (query text, similar document list) 

with open('query_positive_dict.pkl', 'rb') as f: 
    loaded_dict = pickle.load(f)  

In [18]:
query_texts = small_test_queries['queries'].values 

In [19]:
def calc_ranks(df, idx):
    relevant = loaded_dict[query_texts[idx]]
    rank_i = df[df[0]==idx] 
    for i, val in enumerate(rank_i[1].values): 
        if val-1 in relevant: # due to off by one error 
            return i 

In [20]:
### MRR calculation 
ranks = [] 
for i in range(small_test_queries.shape[0]):  
    ranks.append(calc_ranks(rankings, i))  

s = 0 
for r in ranks: 
    if r is None: 
        s += 0 
    else: 
        s += 1/(r+1)  

s = s / len(ranks)

print("MRR for {} queries = {:.3f}".format(len(ranks), s))

MRR for 100 queries = 0.208


# Evaluation: MAP (Mean Average Precision) for 100 queries

In [21]:
def average_precision(df, idx):  
    rank_df = df[df[0] == idx]
    relevant = loaded_dict[query_texts[idx]] 
    correct_predictions, running_sum = 0, 0 
    rank_i = rank_df[1].values 
    for i, val in enumerate(rank_i): 
        if val-1 in relevant: # due to off by one error 
            correct_predictions += 1
            running_sum += correct_predictions / (i+1) 
    return running_sum / len(relevant) 

In [29]:
### MAP calculation 

average_precisions = [] 
for i in range(small_test_queries.shape[0]): 
    average_precisions.append(average_precision(rankings, i)) 
    
    
print("MAP for {} queries = {:.3f}".format(len(average_precisions), np.mean(average_precisions)))

MAP for 100 queries = 0.196
